# Vault Client Count based on Ranjit

[GitHub - WhatsARanjit/vault-counter](https://github.com/WhatsARanjit/vault-counter)

## Overview

Simple container script to run and count HashiCorp Vault entities, roles/users, and tokens created without entities. The script will drill into child namespaces below the supplied namespace input.

This tutorial will also help you start up a Vault server and fill it synthetically with tokens and entities. You can skip straight to the Get Count section.

## Prerequisites

* Software and versions
* Credentials
* Hardware

### Set Main Environment Variables

In [ ]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"
# Some commands may have sensitive information. Prevent commands starting with a space to be saved to shell history.
HISTCONTROL=ignoreboth # do not save lines that begin with space in history

printf "${GREEN}# Set MAIN_DIR for future reference.${NC}\n"
export MAIN_DIR=${PWD}

Be sure to replace `VAULT_ADDR` with your own values.

In [5]:
printf "${GREEN}# Set Vault Variables.${NC}\n"
export VAULT_PORT=8200
export VAULT_TOKEN=root
export VAULT_ADDR=http://localhost:${VAULT_PORT}
# export VAULT_LICENSE=$(cat ../../license/vault.hclic)
# Set dir env vars. Create required directory.
WORK_DIR=config/vault
mkdir -p config/vault/{data,logs,config}

# Set Vault Variables.
cat: ../../license/vault.hclic: No such file or directory


: 1

## Steps

### Start Vault Server in Dev Mode

Start the vault process in the background.

In [ ]:
docker run -d --rm --name vault \
    --cap-add IPC_LOCK \
    -p 8200:8200 \
    -e 'VAULT_DEV_ROOT_TOKEN_ID=root' \
    -e 'VAULT_DEV_LISTEN_ADDRESS=0.0.0.0:8200' \
    hashicorp/vault-enterprise:1.7.5_ent
    # -e "VAULT_ADDR=http://127.0.0.1:8200" \

Configure environment variables for connecting to Vault. We've set the dev mode root token to "`root`".

In [4]:
export VAULT_ADDR="http://127.0.0.1:8200"
export VAULT_TOKEN=root

: 1

Check vault status

In [5]:
vault status

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.7.5+ent
Storage Type    inmem
Cluster Name    vault-cluster-fe242cc0
Cluster ID      20323ab6-2b5e-3336-b71a-4cd9e39b1291
HA Enabled      false


: 1

### Create non-entity tokens

Create some tokens

In [8]:
for i in {1..30}; do
    vault token create -policy=my-policy -policy=other-policy > /dev/null 2>&1
done

: 1

### Userpass

In [53]:
vault auth enable userpass

Success! Enabled userpass auth method at: userpass/


: 1

Create User and login

In [ ]:
for i in {10..19}; do
    vault write auth/userpass/users/user${i} password="password${i}" policies="default"
    vault login -method=userpass username=user${i} password=password${i}
done

Create AppRole but don't login

In [56]:
for i in {20..29}; do
    vault write auth/userpass/users/user${i} password="password${i}" policies="default"
done

Success! Data written to: auth/userpass/users/user20
Success! Data written to: auth/userpass/users/user21
Success! Data written to: auth/userpass/users/user22
Success! Data written to: auth/userpass/users/user23
Success! Data written to: auth/userpass/users/user24
Success! Data written to: auth/userpass/users/user25
Success! Data written to: auth/userpass/users/user26
Success! Data written to: auth/userpass/users/user27
Success! Data written to: auth/userpass/users/user28
Success! Data written to: auth/userpass/users/user29


: 1

### AppRole

Create AppRole and login

In [9]:
vault auth enable approle

Success! Enabled approle auth method at: approle/


: 1

In [ ]:
for i in {10..19}; do
    vault write -f auth/approle/role/app${i}
    vault write auth/approle/login \
    role_id=$(vault read -field=role_id auth/approle/role/app${i}/role-id) \
    secret_id=$(vault write -force -field=secret_id auth/approle/role/app${i}/secret-id)
done

Create AppRole but don't login

In [ ]:
for i in {20..29}; do
    vault write -f auth/approle/role/app${i}
    vault write auth/approle/login \
    role_id=$(vault read -field=role_id auth/approle/role/app${i}/role-id) \
    secret_id=$(vault write -force -field=secret_id auth/approle/role/app${i}/secret-id)
done

### Get Count

In [59]:
docker run --rm \
    -e SKIP_ORPHAN_TOKENS="" \
    -e VAULT_ADDR=http://$(ipconfig getifaddr en0):8200 \
    -e VAULT_TOKEN=$VAULT_TOKEN \
    -e VAULT_NAMESPACE=root \
    -e JSON_OUTPUT="" \
    whatsaranjit/vault_counter:latest

Vault address: http://192.168.17.167:8200[?2004l
Namespace: root
Total entities: 45
Total users/roles: 55
Total tokens: 138
Total orphan tokens: 91


: 1

* `JSON_OUTPUT=""` - Set to anything for json output
* `SKIP_ORPHAN_TOKENS=""` - Set to anything to skip inspecting each token to see if it is tied to an auth method. RECOMMENDED if you have lots of tokens.
* `VAULT_ADDR` - The URL to your Vault server
    * I'm using my Mac's IP for the `counter` container to reach the `vault` container.

**TIPS**
* Start with `SKIP_ORPHAN_TOKENS` enabled to get a feel for impact on system.
* This script can take down a node that has lots of tokens and not enough HW resources.

Sample Output
```shell
Vault address: http://192.168.17.167:8200
Namespace: root/
Total entities: 0
Total users/roles: 30
Total tokens: 91
Total orphan tokens: 91
```

## Conclusion

When you want to purchase that cool Vault Enterprise solution. What one of the key factors for licensing is client count. Luckily for Vault 1.7 and higher, there are metrics built-in that can provide you the client count. For previous versions, there is a tool that can scan your audit logs, if you turned on audit logging. For everyone else, the vault_counter tool can provide you a client count.

## Clean up

### Stop Vault container

In [ ]:
docker stop vault
docker rm vault

### Remove Docker Image

In [60]:
docker image rm whatsaranjit/vault_counter

Untagged: whatsaranjit/vault_counter:latest
Untagged: whatsaranjit/vault_counter@sha256:d5084d45ad330ee3a119d52d76149a1bbc2387355b0884c0da79ab7b6b2a6bb5
Deleted: sha256:e6cdf41d9c7aedd1ebcfcef0a6d0335389e9aaa31baa379ef6dc5927f8032430
Deleted: sha256:2b963b254ecbf4031f4202c59dcae555bea75e7cea5a47a98036b5d5befcb908
Deleted: sha256:7eafc7cd20f8816b2279231a132565647fd79bf979bd0996f82f8269ae5b4174
Deleted: sha256:e0dc89104fe4d9f7d3a3f540a104569b102b7892793f7d7aecb945681845f262
Deleted: sha256:2aa359861e4a4e1e71516dcc22f1add1574fc7c91389f3210e18c26d6341bb06


: 1

In [ ]:
docker image rm vault

## Next steps

### Misc Notes - Please disregard for now

List auth methods

In [ ]:
curl -H "X-Vault-Token: $VAULT_TOKEN" $VAULT_ADDR/v1/sys/auth | jq

In [21]:
curl \
--header "X-Vault-Token: $VAULT_TOKEN" \
--request LIST \
http://127.0.0.1:8200/v1/identity/entity/id

{"errors":[]}04l


: 1

In [23]:
curl -H "X-Vault-Token: $VAULT_TOKEN" --request LIST --request GET $VAULT_ADDR/v1/sys/auth | jq -r '.? | .["data"] | keys[]'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1441  100  1441    0     0   156k      0 --:--:-- --:--:-- --:--:--  156k
approle/
token/


: 1